# BREAKING NEWS: LARGE ASTEROID IS GOING TO OBLITERATE SWANSEA

Size: approximately 40-m radius
Composition: Stony-Iron (typical density, 4500 $kg/m^3$; typical strength, 5 MPa)
Speed: 17 km/s

The asteroid is expected to enter Earth’s atmosphere (100-km altitude) at coordinates (52.7, -2.5), over the midlands, on a bearing of 220 degrees and at an angle of 30 degrees to the horizontal.

In [8]:
from armageddon.solver import Planet
from armageddon.locator import great_circle_distance, PostcodeLocator
from armageddon.damage import damage_zones, impact_risk
from armageddon.mapping import plot_circle, damage_map
from armageddon.extensions import findstrengthradius

In [2]:
earth = Planet()
result = earth.solve_atmospheric_entry(radius=40, angle=30,
                                       strength=5000000, density=4500,
                                       velocity=17000,dt=0.02)
energy = earth.calculate_energy(result)
outcome = earth.analyse_outcome(energy)
outcome

{'outcome': 'Airburst',
 'burst_peak_dedz': 4767.096788480793,
 'burst_altitude': 10994.63059318075,
 'burst_distance': 156270.00516336004,
 'burst_energy': 28009.98811319827}

In [3]:
lat = 52.7
lon = -2.5
bearing = 220
pressures = [1e3, 3.5e3, 27e3, 43e3]
damage = damage_zones(outcome, lat, lon, bearing, pressures)
blat, blon, damrad = damage
damage

(51.61434465235385,
 -3.9548008615362638,
 [184224.6302900536, 67974.37693048915, 16477.630821914834, 11236.96150787215])

In [4]:
damage_map(blat, blon, damrad, lat, lon)

|  Damage Level |  Description    | Pressure (kPa) |
|:-------------:|:---------------:|:--------------:|
|  1  |  ~10% glass windows shatter    |     1.0      |
|  2  | ~90% glass windows shatter     |     3.5      |
|  3  | Wood frame buildings collapse  |     27      |
|  4  | Multistory brick buildings collapse  |     43      |

<p>
<div align="center">Table 1: Pressure thresholds (in kPa) for airblast damage</div>

In [5]:
locator = PostcodeLocator()
postcodes = locator.get_postcodes_by_radius((blat, blon),
                                            radii=damrad)
population = locator.get_population_of_postcode(postcodes)
for pop, zone in zip(population, pressures):
    print("{:5.0f} Pa: {:8.0f}".format(zone, sum(pop)))

 1000 Pa: 12549486
 3500 Pa:  2147307
27000 Pa:   390733
43000 Pa:   273727


In [6]:
means = {'radius': 40, 'angle': 30, 'strength': 5000000,
         'density': 4500, 'velocity': 17000,
         'lat': lat, 'lon': lon, 'bearing': 220.}
# approximate standard deviations taken
stdevs = {'radius': 1, 'angle': 1, 'strength': 500000,
         'density': 450, 'velocity': 1000,
         'lat': 0.025, 'lon': 0.025, 'bearing': 0.5}
impact_risk_df_sector = impact_risk(earth, means, stdevs, pressures, nsamples=1, sector=True)
impact_risk_df_sector

,postcode sector,risk
0,LL250,474.0
1,SY4 1,3120.0
2,BS206,7966.0
3,SY231,7301.0
4,BS408,3833.0
...,...,...
1992,CF142,10875.0
1993,BS308,10295.0
1994,SP1 1,4949.0
1995,TQ138,3836.0


In [9]:
impact_risk_df_postcodes = impact_risk(earth, means, stdevs, pressures, nsamples=1, sector=False)
impact_risk_df_postcodes

,postcode,risk
0,SP8 5EF,14.0
1,SN384NW,0.0
2,TR5 0UE,22.0
3,SA335JS,16.0
4,SY7 9DR,23.0
...,...,...
326717,BS5 9DW,40.0
326718,SN7 7EB,31.0
326719,BS5 9DY,40.0
326720,SA659SG,22.0
